# Install

In [8]:
!pip install spotipy

# Imports

In [9]:
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
import pandas as pd
from __future__ import print_function
import json
import time
import sys

# Credentials from Spotify's API

In [10]:
#client_id= CLIENT_ID
#client_secret= CLIENT_SECRET

pl_id = 'spotify:playlist:5ABHKGoOzxkaa28ttQV9sE'# to be changed
artist_urn = 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'# to be changed

#top50_tracks_2022: 37i9dQZF1DX18jTM2l2fJY
#top50_tracks_2021:
#top50_tracks_2020: 37i9dQZF1DX7Jl5KP2eZaS
#top100_tracks_2019: 37i9dQZF1DWVRSukIED0e9
#top100_tracks_2018: 37i9dQZF1DXe2bobNYDtW8
#top50_tracks_weekly: 37i9dQZEVXbNG2KDcFcKOF
#top50_tracks_daily: 37i9dQZEVXbMDoHDwVN2tF


# Main Features

## Getting main features from playlist

In [12]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id= client_id, client_secret= client_secret))

In [14]:
track_id= [sp.playlist(pl_id)['tracks']['items'][i]['track']['id'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [15]:
name= [sp.playlist(pl_id)['tracks']['items'][i]['track']['name'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [16]:
track_popularity= [sp.playlist(pl_id)['tracks']['items'][i]['track']['popularity'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [17]:
duration_ms= [sp.playlist(pl_id)['tracks']['items'][i]['track']['duration_ms'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [ ]:
#explicit= [sp.playlist(pl_id)['tracks']['items'][i]['track']['explicit'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [18]:
artist_name= [sp.playlist(pl_id)['tracks']['items'][i]['track']['artists'][0]['name'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [19]:
artist_id= [sp.playlist(pl_id)['tracks']['items'][i]['track']['artists'][0]['id'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [20]:
release_date= [sp.playlist(pl_id)['tracks']['items'][i]['track']['album']['release_date'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [21]:
artist_urn= [sp.playlist(pl_id)['tracks']['items'][i]['track']['artists'][0]['uri'] for i in range(len(sp.playlist(pl_id)['tracks']['items']))]

In [22]:
followers= [sp.artist(i)['followers']['total'] for i in artist_urn]

In [23]:
artist_popularity= [sp.artist(i)['popularity'] for i in artist_urn]

## Features into dataframe

In [30]:
main_df= pd.DataFrame({'track_id': track_id,
                  'track_name': name,
                  'track_popularity': track_popularity,
                  'release_date': release_date,
                  'duration_ms': duration_ms,
                  'artists': artist_name,
                  'artists_ids': artist_id,
                  'artist_popularity' : artist_popularity, 
                  'followers': followers,}                      ,
                  columns=['track_id','track_name','track_popularity','release_date','duration_ms','artists','artists_ids','artist_popularity', 'followers'])
main_df

,track_id,track_name,track_popularity,release_date,duration_ms,artists,artists_ids,artist_popularity,followers
0,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,90,2020-03-20,200040,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,99,61125013
1,7qiZfU4dY1lWllzX7mPBI3,Shape of You,85,2017-03-03,233712,Ed Sheeran,6eUKZXaKkcviH0Ku9w2n3V,92,110567101
2,2XU0oxnq2qxCpomAAuJY8K,Dance Monkey,76,2019-10-17,209438,Tones And I,2NjfBq1NflQcKSeiDooVjY,73,3000740
3,7qEHsqek33rTcFNT9PFqLf,Someone You Loved,86,2019-05-17,182160,Lewis Capaldi,4GNC7GD6oZMSxPGyXy4MNB,83,10687418
4,7wGoVu4Dady5GV0Sv4UIsx,rockstar,0,2017-09-15,218320,Post Malone,246dkjvS1zLTtiykXe5h60,90,40046028
...,...,...,...,...,...,...,...,...,...
95,7ygpwy2qP3NbrxVkHvUhXY,Wonderwall - Remastered,75,1995,258773,Oasis,2DaxqgrOhkeH0fpeiQq2f4,78,8426676
96,68Dni7IE4VyPkTOH9mRWHr,No Role Modelz,87,2014-12-09,292799,J. Cole,6l3HvQ5sa6mXTsMTB19rO5,89,19556686
97,0KKkJNfGyhkQ5aFogxQAPU,That's What I Like,82,2016-11-17,206693,Bruno Mars,0du5cEVh5yTK9QJze8zA0C,90,47052216
98,0nbXyq5TXYPCO7pr3N8S4I,The Box,80,2019-12-06,196652,Roddy Ricch,757aE44tKEUQEqRuT6GnEB,81,9092716


# Audio Features

## Getting audio features from playlist

In [61]:
tracks=[]

for track_id in main_df.loc[:, 'track_id']:
    track=f'spotify:track:{track_id}'
    audio_features= sp.audio_features(track)
    tracks.append(audio_features[0])

In [66]:
tracks

[{'danceability': 0.514,
  'energy': 0.73,
  'key': 1,
  'loudness': -5.934,
  'mode': 1,
  'speechiness': 0.0598,
  'acousticness': 0.00146,
  'instrumentalness': 9.54e-05,
  'liveness': 0.0897,
  'valence': 0.334,
  'tempo': 171.005,
  'type': 'audio_features',
  'id': '0VjIjW4GlUZAMYd2vXMi3b',
  'uri': 'spotify:track:0VjIjW4GlUZAMYd2vXMi3b',
  'track_href': 'https://api.spotify.com/v1/tracks/0VjIjW4GlUZAMYd2vXMi3b',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0VjIjW4GlUZAMYd2vXMi3b',
  'duration_ms': 200040,
  'time_signature': 4},
 {'danceability': 0.825,
  'energy': 0.652,
  'key': 1,
  'loudness': -3.183,
  'mode': 0,
  'speechiness': 0.0802,
  'acousticness': 0.581,
  'instrumentalness': 0,
  'liveness': 0.0931,
  'valence': 0.931,
  'tempo': 95.977,
  'type': 'audio_features',
  'id': '7qiZfU4dY1lWllzX7mPBI3',
  'uri': 'spotify:track:7qiZfU4dY1lWllzX7mPBI3',
  'track_href': 'https://api.spotify.com/v1/tracks/7qiZfU4dY1lWllzX7mPBI3',
  'analysis_url': 'https://a

In [67]:
audio_df = pd.DataFrame(tracks)
audio_df

AttributeError: 'NoneType' object has no attribute 'keys'

## Audio features into a clean dataframe

In [42]:
audio_df= pd.DataFrame(tracks)

ValueError: If using all scalar values, you must pass an index

In [ ]:
audio_df= audio_df.drop(['type','uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], axis=1)

In [ ]:
audio_df= audio_df.rename(columns={"id": "track_id"})
audio_df

# Combining the 2 dataframes

In [ ]:
spotify_df= main_df.merge(audio_df, on='track_id')
spotify_df

# Adding preview_url

In [ ]:
previews=[]

for track_id in spotify_df.loc[:, 'track_id']:
    urn=f'spotify:track:{track_id}'
    if sp.track(urn)['preview_url']:
        preview_url= sp.track(urn)['preview_url']
        previews.append(preview_url)
    else:
        previews.append('### no audio preview available ###')
previews

In [ ]:
preview_df= pd.DataFrame(previews, columns=['preview_url'])
preview_df

# Adding cover_url

In [ ]:
covers=[]

for track_id in spotify_df.loc[:, 'track_id']:
    urn=f'spotify:track:{track_id}'
    if sp.track(urn)['album']['images'][0]['url']:
        cover_url= sp.track(urn)['album']['images'][0]['url']
        covers.append(cover_url)
    else:
        covers.append('### no cover available ###')
covers

In [ ]:
covers_df= pd.DataFrame(covers, columns=['cover_url'])
covers_df

# Combining all dataframe in one

In [ ]:
playlist_df = pd.concat([spotify_df, preview_df, covers_df ], axis=1)
playlist_df

# Export as csv

In [ ]:
playlist_df.to_csv('playlist_df.csv')